In [519]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

In [521]:
df = pd.read_csv('Сбер.csv')
not_dop_office = df[df['Тип'] != 'Доп офис']
dop_office = df[df['Тип'] == 'Доп офис']
df.shape, dop_office.shape

<class 'pandas.errors.ParserError'>: Error tokenizing data. C error: EOF inside string starting at row 4908

(3639, 6)

In [509]:
# (первый 
# первый этаж
# ,(комната
# "инв",

In [510]:
s = 'saf (000) sfaf'
s.find("(")

4

In [511]:
excluded = ["пом", "(Помещения", " (ком.", "литер", "Литер", "(", "офис", 
     "первый этаж", "второй этаж", "третрий этаж", "цокольный этаж", "1 эт", "2 эт", "3 эт",
    "литер", "каб", "блок", "напротив", "комн", " оф.",  "б/н", "цок.",  "пом.",
           "подвальный этаж", "нежилые пом", "аптеки", "гостиница","кв.", "инв.", "встроенное",
           "часть нежилого", "Военный городок",  "часть", "КОЦ", "нежилые", "в 15 метрах",
           "номер на"] 

In [512]:
dop_office['Адрес'] = dop_office['Адрес'].apply(lambda s: s[:s.find("(")] + s[s.find(")")+1 :]
                                                if s.find("(") != -1 and s.find(")") != -1 else s)

for e in excluded:

    dop_office['Адрес'] = dop_office['Адрес'].str.replace("№", "")
    dop_office['Адрес'] = dop_office['Адрес'].str.replace(" №", "")
    dop_office['Адрес'] = dop_office['Адрес'].str.replace(" № ", "")
    dop_office['Адрес'] = dop_office['Адрес'].str.replace("здание", "")
    dop_office['Адрес'] = dop_office['Адрес'].str.replace("Здание", "")
    dop_office['Адрес'] = dop_office['Адрес'].str.replace("зд.", "")
    dop_office['Адрес'] = dop_office['Адрес'].str.replace("строение", "стр.")
    dop_office['Адрес'] = dop_office['Адрес'].str.replace("строен.", "стр.")
    dop_office['Адрес'] = dop_office['Адрес'].str.replace("з/у", "")
    dop_office['Адрес'] = dop_office['Адрес'].str.replace("  ", " ")
    dop_office['Адрес'] = dop_office['Адрес'].str.replace("   ", " ")

    dop_office['Адрес'] = dop_office['Адрес'].apply(lambda s: s[:s.find(e)] if s.find(e) != -1 else s)
    e_plus = ', ' + e 
    dop_office['Адрес'] = dop_office['Адрес'].apply(lambda s: s[:s.find(e_plus)] if s.find(e_plus) != -1 else s)
    e_plus = ',' + e 
    dop_office['Адрес'] = dop_office['Адрес'].apply(lambda s: s[:s.find(e_plus)] if s.find(e_plus) != -1 else s)

for row in dop_office.itertuples():
  
    if row.Адрес[-1] == ',' or row.Адрес[-1] == ';' or row.Адрес[-1] == '.' or row.Адрес[-1] == ')':
        dop_office.loc[row.Index, 'Адрес'] = row.Адрес[:-1]
    elif row.Адрес[-2:] == ' ,' or row.Адрес[-2:] == '. ' or row.Адрес[-2:] == '; ' or row.Адрес[-2:] == ', ' or row.Адрес[-2:] == '; ':
        dop_office.loc[row.Index, 'Адрес'] = row.Адрес[:-2]
    elif row.Адрес[-3:] == '  ,' or row.Адрес[-3:] == ' , ' or row.Адрес[-3:] == ',  ':
        dop_office.loc[row.Index, 'Адрес'] = row.Адрес[:-3]

In [513]:
ex = '№8598/0244'
df[df['Номер ТП.1'] == ex]['Адрес'].values

array(['450056, Республика Башкортостан, Уфимский район, с.Булгаково, микрорайон Аэропорт'],
      dtype=object)

In [514]:
dop_office[dop_office['Номер ТП.1'] == ex]['Адрес'].values

array(['450056, Республика Башкортостан, Уфимский район, с.Булгаково, микрорайон Аэропорт'],
      dtype=object)

In [515]:
dop_office[['Адрес', 'Номер ТП.1']].sample(15).values

array([['162700, Вологодская обл., Череповецкий р-н, Судский п/с, п. Суда, ул. Гагарина, д. 30',
        '№ 8638/0174'],
       ['601810, Владимирская область, Юрьев-Польский район, с. Небылое, ул. Первомайская, д. 2',
        '8611/0414'],
       ['385061, Республика Адыгея , г. Майкоп, ст-ца Ханская, ул. Верещагина, д. 105',
        '8620/015'],
       ['358000, Республика Калмыкия, г.Элиста, ул.В.И.Ленина, 305',
        '№ 8579/022 кассово-инкассаторский центр "Элистинский"'],
       ['242611, Брянская область, г. Фокино, ул. Карла Маркса, д. 1',
        '8605/0164'],
       ['649750, Республика Алтай, Улаганский район, с.Улаган, ул.А.В.Санаа, 16',
        '№8558/018'],
       ['453147, Республика Башкортостан, Стерлитамакский р-н, с. Октябрьское, ул. Мира, д. 9',
        '8598/0385'],
       ['658616, Алтайский край, Завьяловский район, с. Харитоново, ул. Советская, 21',
        '8644/0619'],
       ['414000, Астраханская обл., г. Астрахань, ул. Кирова, д. 30',
        '8625/0142']

In [502]:
res = pd.concat([not_dop_office, dop_office])

In [516]:
res.shape

(3639, 6)

In [505]:
res.to_csv("Сбер_без_пом_и_пр.csv", index=False)

In [504]:
dop_office.to_csv("dop_office.csv")